In [1]:
import pandas as pd
import psycopg2

In [6]:
conn = psycopg2.connect(host="35.200.234.61",database="sales", user="postgres", password="cx6ac54nmgGtLD1y")

In [7]:
# This query gets the snippets and caught 
#signals along with the ids of signal and snippets for byju's
sql="""select
	snippet.task_id,
	signal_caught.snippet_id,
	signal_caught.signal_generated_id,
	snippet.text_ as snippet_text,
	signal_generated.text_ as signal_text,
	product_signal."name",
	signal_caught.score,
	signal_caught.match_method
from
	signal_caught
left join signal_generated on
	signal_generated.id = signal_caught.signal_generated_id
left join product_signal on
	product_signal.id = signal_generated.signal_id
left join snippet on
	snippet.id = signal_caught.snippet_id
where
	signal_caught.snippet_id in (
	select
		snippet.id
	from
		snippet
	where
		snippet.task_id in(select
		task.id
	from
		task
	where
		task."owner" in (
		select
			org_user.userid
		from
			org_user
		where
			org_user.organizationid in (
			select
				organization.id
			from
				organization
			where
				organization."name" like '%Byju%'))))
and signal_caught.match_method = 'USE'
order by snippet.id asc"""

In [8]:
df_caught_signals=pd.read_sql_query(sql,conn)

In [9]:
len(df_caught_signals)

0

In [8]:
"""displaying the countof unique values for aal the columns 
and the name of unique columns for signal names"""

for row in df_caught_signals.columns.to_list():
    print(str(row)+"-->"+str(len(df_caught_signals[row].unique())))
    if row in 'name':
        print(df_caught_signals[row].unique())

task_id-->124
snippet_id-->276
signal_generated_id-->64
snippet_text-->82
signal_text-->27
name-->10
['Value Proposition' 'Lead Qualification' 'Introduction'
 'Feature Explanation' 'Objection Handling' 'Deal Finalization'
 'Next Steps' 'Feature Need Mapping' 'Competitor Mapping'
 'Value Prop Signal']
score-->98
match_method-->1


In [3]:
# Here i am getting all the snippets for byju's
sql_snippets="""select
    snippet.id as id,
	snippet.speaker,
	snippet.text_,
    snippet.task_id,
    snippet.from_time,
    snippet.to_time
from
	snippet
where
	snippet.task_id in (
	select
		task.id
	from
		task
	where
		task."owner" in (
		select
			org_user.userid
		from
			org_user
		where
			org_user.organizationid in (
			select
				organization.id
			from
				organization
			where
				organization."name" like '%Byju%')))
order by snippet.id asc"""

In [4]:
df_snippets=pd.read_sql_query(sql_snippets,conn)

In [5]:
df_snippets.rename(columns={"text_":"text"},inplace=True)

In [6]:
# making data for new application
df_snippets.to_excel("/home/andy/Desktop/byjoose_snippets.xlsx",index=None)

# making data for the old system

In [21]:
df_caught_signals_new = df_caught_signals[["snippet_id","signal_generated_id","name","score","match_method","signal_text"]]

In [22]:
df_caught_signals_new.columns

Index(['snippet_id', 'signal_generated_id', 'name', 'score', 'match_method',
       'signal_text'],
      dtype='object')

In [23]:
out=df_snippets.merge(df_caught_signals_new,on="snippet_id",how="left")

In [24]:
out.to_excel("/home/andy/Desktop/xyz.xlsx")